In [10]:
import akshare as ak
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

code_list = ['sh000300', "sz399905"]

In [11]:
index_data = {}
for code in code_list:
    df = ak.stock_zh_index_daily(symbol=code)
    df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df = df[df["date"] >= "2016-01-01"]
    index_data[code] = {"code": code, "data": df}

In [12]:
import talib

In [13]:
for code in code_list:
    j = index_data[code]
    df = j["data"]
    close = df["close"].to_numpy()
    j["close"] = close
    j["open"] = df["open"].to_numpy()
    j["rsi2"] = talib.RSI(close, timeperiod=2)

In [14]:
dates = index_data["sh000300"]["data"]["date"].to_numpy()

In [10]:
# 单独测试RSI
account = 1000000.0
profits = []
r_stop = 75
cur_code = ""
amount = 0
prev_account = 0
for (index, date) in enumerate(dates):
    for code in code_list:
        # 昨天收盘RSI2满足条件，第二天开盘开干
        rsi2 = index_data[code]["rsi2"][index]
        close = index_data[code]["close"][index]
        if rsi2 < 5 and cur_code == "":
            cur_code = code
            amount = account / close
            prev_account = account
            account = 0
            print (date, "buy ", code)
        if rsi2 >= r_stop and cur_code == code:
            account = amount * close
            cur_code = ""
            amount = 0
            profit = account - prev_account
            print (date, "sell ", code, profit)
            profits.append(profit)

2016-01-28 buy  sh000300
2016-02-04 sell  sh000300 45905.816755181644
2016-02-29 buy  sz399905
2016-03-02 sell  sz399905 78722.7568994863
2016-04-21 buy  sz399905
2016-05-03 sell  sz399905 37063.67441695533
2016-05-09 buy  sh000300
2016-05-23 sell  sh000300 8188.192612683633
2016-06-13 buy  sh000300
2016-06-22 sell  sh000300 25798.60341660399
2016-08-26 buy  sh000300
2016-08-31 sell  sh000300 7485.535298121627
2016-09-12 buy  sh000300
2016-09-22 sell  sh000300 10516.337352226255
2016-12-12 buy  sz399905
2016-12-21 sell  sz399905 19135.674182507675
2017-01-13 buy  sz399905
2017-01-23 sell  sz399905 -4475.998182093725
2017-03-30 buy  sz399905
2017-04-05 sell  sz399905 31638.746603198815
2017-04-18 buy  sh000300
2017-05-12 sell  sh000300 -28150.238585378742
2017-05-23 buy  sz399905
2017-06-06 sell  sz399905 16377.803284351481
2017-07-17 buy  sz399905
2017-07-20 sell  sz399905 47329.812629646156
2017-08-11 buy  sh000300
2017-08-17 sell  sh000300 26258.829795402242
2017-10-19 buy  sz399905


In [11]:
print ('total account = ', account)
print('total len = ', len(profits))
winning_len = len(list(filter(lambda x: x > 0, profits)))
print('winning len = ', winning_len)
print("winning rate = ", winning_len * 1.0 / len(profits))
print("avg profit = ", sum(profits) / len(profits))
print("max loss = ", min(profits))

total account =  1784105.5440679227
total len =  83
winning len =  60
winning rate =  0.7228915662650602
avg profit =  9447.054747806298
max loss =  -118679.74805991584


In [17]:
# 累积RSI
account = 1000000.0
profits = []
r_stop = 65
cur_code = ""
amount = 0
prev_account = 0
for (index, date) in enumerate(dates):
    for code in code_list:
        # 昨天收盘RSI2满足条件，第二天开盘开干
        rsi2 = index_data[code]["rsi2"][index]
        cu_rsi = rsi2 + index_data[code]["rsi2"][index-1] + index_data[code]["rsi2"][index-2]
        close = index_data[code]["close"][index]
        if cu_rsi < 65 and cur_code == "":
            cur_code = code
            amount = account / close
            prev_account = account
            account = 0
            print (date, "buy ", code)
        if rsi2 >= r_stop and cur_code == code:
            account = amount * close
            cur_code = ""
            amount = 0
            profit = account - prev_account
            print (date, "sell ", code, profit)
            profits.append(profit)
if cur_code != "":
     account = amount * index_data[cur_code]["close"][-1]
     cur_code = ""
     amount = 0
     profit = account - prev_account
     print (date, "sell ", cur_code, profit)
     profits.append(profit)

2016-01-11 buy  sz399905
2016-01-19 sell  sz399905 11167.048697136808
2016-01-28 buy  sh000300
2016-02-02 sell  sh000300 38117.95008569094
2016-02-29 buy  sh000300
2016-03-02 sell  sh000300 63401.24338467652
2016-03-11 buy  sh000300
2016-03-14 sell  sh000300 17476.15664933878
2016-04-21 buy  sh000300
2016-05-03 sell  sh000300 18929.111598694
2016-05-10 buy  sh000300
2016-05-16 sell  sh000300 9808.293180032633
2016-06-14 buy  sh000300
2016-06-22 sell  sh000300 21843.64894502447
2016-07-20 buy  sh000300
2016-07-26 sell  sh000300 11664.090536864707
2016-07-29 buy  sz399905
2016-08-04 sell  sz399905 6793.361897130264
2016-08-24 buy  sh000300
2016-08-31 sell  sh000300 -745.1197458852548
2016-09-13 buy  sh000300
2016-09-21 sell  sh000300 2316.9003946210723
2016-10-31 buy  sh000300
2016-11-01 sell  sh000300 8196.67819719622
2016-12-06 buy  sh000300
2016-12-09 sell  sh000300 12074.130462546134
2016-12-13 buy  sz399905
2016-12-16 sell  sz399905 7846.702009747038
2016-12-19 buy  sh000300
2017-01

In [18]:
print ('total account = ', account)
print('total len = ', len(profits))
winning_len = len(list(filter(lambda x: x > 0, profits)))
print('winning len = ', winning_len)
print("winning rate = ", winning_len * 1.0 / len(profits))
print("avg profit = ", sum(profits) / len(profits))
print("max loss = ", min(profits))

total account =  2069890.5982258029
total len =  140
winning len =  105
winning rate =  0.75
avg profit =  7642.075701612877
max loss =  -124231.11328728194
